In [24]:
import sys,os,dlib,glob,numpy
from skimage import io
import cv2
import imutils
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [25]:
#face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 沒用到

In [26]:
yourPath = 'images/' #讀取路徑
imgPath='text/'
allFileList = os.listdir(yourPath)
allimg = os.listdir(imgPath)
fet=[]
M=[]
for file in allFileList:
    if os.path.isdir(os.path.join(yourPath,file)):
        fet.append(file)
for W in allimg:
    M.append(W)

In [30]:
### Feature extraction and store #########
featList=[]
probeList=[]
imgList=[]
probe_id = []
sift = cv2.xfeatures2d.SIFT_create() #載入sift
#========= features =================
#將人連做特徵擷取
for im in M: 
    gray = cv2.imread(imgPath+im)
    gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)
    kps1, des1 = sift.detectAndCompute(gray, None)
    featList.append(des1)
######################
pid = 0
#將資料夾內的人臉做特徵擷取
for i in range(len(fet)):
    X=os.listdir(yourPath+"/"+fet[i])
    for j in X:
        img2 = cv2.imread(yourPath+fet[i]+"/"+j)
        gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
        kps2, des2 = sift.detectAndCompute(gray, None)
    
        probeList.append(des2)
        pid = int(os.path.basename(j.replace('.jpg','')))
        probe_id.append(pid)
    
probe_id= np.array(probe_id)

In [45]:
Anslist=[]
bf = cv2.BFMatcher() #載入BFMatcher
# BFMatcher with default params
for s in range(100):    
    distList= []
    for ct, feat in enumerate(probeList):
        des1 = featList[s]
        des2 = feat
        matches = bf.knnMatch(des1,des2, k=2) #用knnMatch做比對
        good=[]
        for m,n in matches:
            if m.distance < 0.85*n.distance:
                good.append([m])

        total_dist = 0
        for m in matches:
            total_dist += m[0].distance
        if len(matches)>0:
            total_dist = total_dist/len(matches)
        else:
            total_dist = sys.maxint
        
        distList.append(total_dist)
    
    
    mid = distList.index(min(distList))
    count=-1

    for i in range(mid+1):
        if probe_id[i]==1:
            count+=1
    
    Anslist.append(count)


In [49]:
print(Anslist)
df = pd.DataFrame(Anslist)

df.columns=['Category']
df.to_csv("C.csv",index_label='id')

[81, 10, 98, 94, 62, 76, 63, 68, 43, 86, 76, 65, 76, 76, 47, 65, 34, 65, 7, 90, 76, 65, 76, 76, 91, 59, 76, 76, 83, 81, 39, 61, 76, 99, 6, 75, 10, 13, 30, 76, 76, 65, 76, 0, 12, 79, 40, 47, 76, 76, 81, 79, 10, 48, 53, 31, 18, 76, 81, 12, 76, 81, 76, 76, 84, 74, 76, 33, 82, 41, 81, 55, 76, 81, 12, 76, 76, 76, 65, 47, 98, 80, 67, 10, 65, 76, 37, 81, 4, 26, 81, 36, 76, 10, 3, 69, 80, 87, 81, 95]
